In [1]:
from data_utils import get_CIFAR10_data
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, Merge, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
import tensorflow_weave
reload(tensorflow_weave)
from tensorflow_weave import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [49]:
filter_size = 3
num_filters = 40
pad_size = filter_size//2
mid_layer = 20


X_train = data['X_train']
Y_train = data['y_train']
X_Val = data['X_val']
Y_Val = data['y_val']

Y_train = to_categorical(Y_train, num_classes=10)
Y_val = to_categorical(Y_Val, num_classes=10)

In [50]:
inputs = Input(shape=(3,32,32))

# x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

x_per = Conv2D(num_filters,
               kernel_size = (filter_size,filter_size),
               strides=(1,1),
               padding='same',
               activation='relu')(inputs)

x_loc = Conv2D(num_filters,
               kernel_size= (filter_size,filter_size),
               strides=(1,1),
               padding='same',
               activation = 'relu')(inputs)

x_zero = ZeroWeave()(x_loc)
x_weave = ArrayWeave()(x_per)
x = Add()([x_weave, x_zero])

# x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

x = Conv2D(num_filters,
           kernel_size= (filter_size,filter_size),
           strides=(filter_size,filter_size),
           padding='same',
           activation = 'relu')(x)

x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(mid_layer, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
conv2d_82 (Conv2D)              (None, 40, 32, 32)   1120        input_22[0][0]                   
__________________________________________________________________________________________________
conv2d_83 (Conv2D)              (None, 40, 32, 32)   1120        input_22[0][0]                   
__________________________________________________________________________________________________
array_weave_19 (ArrayWeave)     (None, 40, 94, 94)   0           conv2d_82[0][0]                  
__________________________________________________________________________________________________
zero_weave

In [51]:
hist1 = model.fit(X_Val, Y_val, epochs = 10, verbose=1)

Epoch 1/10
1000/1000 [==============================] - 60s 60ms/step - loss: 14.6901 - acc: 0.0840
Epoch 2/10
1000/1000 [==============================] - 52s 52ms/step - loss: 14.7158 - acc: 0.0870
Epoch 3/10
 224/1000 [=====>........................] - ETA: 42s - loss: 14.3912 - acc: 0.1071

KeyboardInterrupt: 

In [43]:
inputs = Input(shape=(3,32,32))

x = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

x = Conv2D(num_filters,
               kernel_size = (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation='relu')(x)

x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

x = Conv2D(num_filters,
               kernel_size= (filter_size,filter_size),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x)

x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

x = Conv2D(3*num_filters,
           kernel_size= (filter_size,filter_size),
           strides=(1,1),
           padding='valid',
           activation = 'relu')(x)

x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(mid_layer, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 3, 32, 32)         0         
_________________________________________________________________
zero_padding2d_41 (ZeroPaddi (None, 3, 34, 34)         0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 10, 32, 32)        280       
_________________________________________________________________
zero_padding2d_42 (ZeroPaddi (None, 10, 34, 34)        0         
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 10, 32, 32)        910       
_________________________________________________________________
zero_padding2d_43 (ZeroPaddi (None, 10, 34, 34)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 30, 32, 32)        2730      
__________

In [44]:
hist1 = model.fit(X_train, Y_train, epochs = 10, verbose=1)

Epoch 1/10
49000/49000 [==============================] - 403s 8ms/step - loss: 2.8298 - acc: 0.0990
Epoch 2/10
49000/49000 [==============================] - 391s 8ms/step - loss: 2.3028 - acc: 0.0974
Epoch 3/10
49000/49000 [==============================] - 394s 8ms/step - loss: 2.3028 - acc: 0.0979
Epoch 4/10
28288/49000 [================>.............] - ETA: 2:55 - loss: 2.3028 - acc: 0.0967

KeyboardInterrupt: 